In [1]:
from sklearn.model_selection import KFold, train_test_split
import time
import orjson
import sys
import shutil
import os.path
import os
import random
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import torch
from random import shuffle
from pymatgen.core import Structure
sys.path.append("../../")
from utils.save_and_load import load_from_json, save_to_json
from utils.experiment_tracking import log_mean_std_based_on_test_metrics
from aim import Run
from aim.pytorch_lightning import AimLogger

In [2]:
!pip install -e ./

Obtaining file:///home/danya/mof/shg_ml/gnn_cmp/MOFTransformer
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [3 lines of output]
      error in moftransformer setup command: 'install_requires' must be a string or list of strings containing valid project/version requirement specifiers; Expected package name at the start of dependency specifier
          -e git+ssh://git@github.com/ricet8ur/shg-ml.git@a14db624e5e7d39d476f43135313d30bcfe1350e#egg=moftransformer&subdirectory=gnn_cmp/MOFTransformer
          ^
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
import moftransformer
from moftransformer.examples import example_path
from moftransformer.utils import prepare_data
from pathlib import Path
from moftransformer import __root_dir__
import moftransformer.global_aim

/home/danya/miniconda3/envs/moft/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [4]:
def finetune(
    dataset_name: str = "dataset_of_max_abs_shg",
    random_seed: int = 42,
):
    experiment_name = f"MOFTransformer FT from pmtransformer on {dataset_name}"
    run = Run(
        experiment=experiment_name,
        log_system_params=True,
    )
    moftransformer.global_aim.global_aim = run
    my_path = {
        "root_cifs": os.path.join(__root_dir__, f"trained/{dataset_name}/raw"),
        "root_dataset": os.path.join(__root_dir__, f"trained/{dataset_name}/dataset"),
        "data_root": os.path.join(__root_dir__, f"trained/{dataset_name}/dataset"),
        "downstream": "shg",
    }
    os.makedirs(
        os.path.join(__root_dir__, f"trained/{dataset_name}/raw"), exist_ok=True
    )
    os.makedirs(
        os.path.join(__root_dir__, f"trained/{dataset_name}/dataset"), exist_ok=True
    )
    # Get example path
    root_cifs = my_path["root_cifs"]
    root_dataset = my_path["root_dataset"]
    downstream = my_path["downstream"]

    train_fraction = 0.8  # default value
    test_fraction = 0.1  # default value

    # Run prepare data

    # data root and downstream from example
    log_dir = "./logs/"
    # load_path = "pmtransformer" (default)

    dataset_shg = load_from_json(f"../../data/final_data/{dataset_name}.json")

    dataset_keys = [k for k, v in dataset_shg.items()]
    # create dataset
    target_dataset = {}
    for k, v in dataset_shg.items():
        Structure.from_dict(v["structure"]).to(
            filename=str(Path(root_cifs) / f"{k}.cif")
        )
        target_dataset[k] = v["shg"]

    save_to_json(target_dataset, str(Path(root_cifs) / f"raw_{downstream}.json"))

    # enable deterministic learning
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    random.seed(random_seed)
    np.random.seed(random_seed)
    %env CUBLAS_WORKSPACE_CONFIG=:4096:8

    # aim_logger = AimLogger(repo="./", experiment=experiment_name)
    
    k_folds = 10
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=random_seed)
    data_range = np.arange(0, len(dataset_keys))

    fold_partition = {}
    fold_partition_names = {}
    crystal_predictions_log_for_all_folds = {}
    run.set_artifacts_uri(f"file://{os.getcwd()}/artifacts/")
    for fold, (train_val_idx, test_idx) in enumerate(kfold.split(data_range)):
        print(f"FOLD {fold}")

        train_idx, val_idx = train_test_split(
            train_val_idx, train_size=8 / 9, random_state=random_seed
        )

        # dataset_key_target = {
        #     dataset_keys[idx]: v["total_energy"].data.cpu().item()
        #     for idx, v in enumerate(dataset)
        # }
        # dataset_key_index = {
        #     dataset_keys[idx]: v["total_energy"].data.cpu().item()
        #     for idx, v in enumerate(dataset)
        # }
        # dataset_index_key_target = {
        #     idx: dict(target=v["total_energy"].data.cpu().item(), key=dataset_keys[idx])
        #     for idx, v in enumerate(dataset)
        # }
        dataset_info = dict()
        prepare_data(
            root_cifs,
            root_dataset,
            downstream=downstream,
            # train_fraction=train_fraction,
            # test_fraction=test_fraction,
            train_idx=train_idx,
            val_idx=val_idx,
            test_idx=test_idx,
            dataset_info=dataset_info,
        )
        # g = torch.manual_seed(random_seed)

        print(f"# graphs for training: {len(train_idx)}")
        print(f"# graphs for validation: {len(val_idx)}")

        # log fold partition
        fold_partition = dict(
            fold=fold_partition.get("fold", {})
            | {
                fold: dict(
                    train_idx=train_idx.tolist(),
                    val_idx=val_idx.tolist(),
                    test_idx=test_idx.tolist(),
                )
            }
        )
        fold_partition_names = dict(
            fold=fold_partition_names.get("fold", {})
            | {
                fold: dict(
                    train_names=[dataset_keys[x] for x in train_idx.tolist()],
                    val_names=[dataset_keys[x] for x in val_idx.tolist()],
                    test_names=[dataset_keys[x] for x in test_idx.tolist()],
                )
            }
        )

        # run['dataset_index_key_target'] = dataset_index_key_target
        hparams = dict(
            root_dataset=root_dataset,
            downstream=downstream,
            log_dir=log_dir,
            max_epochs=50,
            batch_size=8,
            mean=None,
            std=None,
        )

        aux_dict = dict(
            dataset_key_index=dataset_info["dataset_key_index"],
            dataset_index_key=dataset_info["dataset_index_key"],
        )
        # print(f"{run=}")
        moftransformer.run(aux_dict=aux_dict, fold=fold, **hparams)

        aux_dict = dict(
            dataset_key_target=target_dataset,
            dataset_key_index=aux_dict["dataset_key_index"],
            dataset_index_key=aux_dict["dataset_index_key"],
        )

        run["run_params"] = hparams
        run["hparams"] = hparams

        crystal_predictions_log_for_all_folds = dict(
            fold=crystal_predictions_log_for_all_folds.get("fold", {})
            | {fold: moftransformer.global_aim.crystal_predictions_log.copy()}
        )


    log_mean_std_based_on_test_metrics(run)
    jsons_to_log = dict(
        fold_partition=fold_partition,
        fold_partition_names=fold_partition_names,
        dataset_key_target=target_dataset,
        dataset_key_index=aux_dict["dataset_key_index"],
        dataset_index_key=aux_dict["dataset_index_key"],
        crystal_predictions_log_for_all_folds=crystal_predictions_log_for_all_folds,
    )
    json_name = "train_info.json"
    save_to_json(jsons_to_log, json_name)
    run.log_artifact(json_name)
    run.close()

In [5]:
# finetune()

In [ ]:
finetune('base_dataset_of_eff_shg')

/home/danya/miniconda3/envs/moft/lib/python3.10/site-packages/pymatgen/core/structure.py:2943: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Co1', 'Co1', 'Co1', 'Co1', 'Co2', 'Co2', 'Co2', 'Co2', 'Ge1', 'Ge1', 'Ge1', 'Ge1', 'Ge2', 'Ge2', 'Ge2', 'Ge2', 'C1', 'C1', 'C1', 'C1', 'C2', 'C2', 'C2', 'C2', 'C3', 'C3', 'C3', 'C3', 'C4', 'C4', 'C4', 'C4', 'C5', 'C5', 'C5', 'C5', 'C6', 'C6', 'C6', 'C6', 'C7', 'C7', 'C7', 'C7', 'C8', 'C8', 'C8', 'C8', 'Cl1', 'Cl1', 'Cl1', 'Cl1', 'Cl2', 'Cl2', 'Cl2', 'Cl2', 'Cl3', 'Cl3', 'Cl3', 'Cl3', 'Cl4', 'Cl4', 'Cl4', 'Cl4', 'Cl5', 'Cl5', 'Cl5', 'Cl5', 'Cl6', 'Cl6', 'Cl6', 'Cl6', 'O1', 'O1', 'O1', 'O1', 'O2', 'O2', 'O2', 'O2', 'O3', 'O3', 'O3', 'O3', 'O4', 'O4', 'O4', 'O4', 'O5', 'O5', 'O5', 'O5', 'O6', 'O6', 'O6', 'O6', 'O7', 'O7', 'O7', 'O7', 'O8', 'O8', 'O8', 'O8']`.
  writer: Any = CifWriter(self, **kwargs)
/home/danya/miniconda3/envs/m

env: CUBLAS_WORKSPACE_CONFIG=:4096:8
FOLD 0


100%|██████████| 974/974 [00:00<00:00, 26095.51it/s]
Global seed set to 42


# graphs for training: 778
# graphs for validation: 98
load model : /home/danya/mof/shg_ml/gnn_cmp/MOFTransformer/moftransformer/database/pmtransformer.ckpt
num_device 1


/home/danya/miniconda3/envs/moft/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:159: UserWarning: You passed `deterministic=True` and `benchmark=True`. Note that PyTorch ignores torch.backends.cudnn.deterministic=True when torch.backends.cudnn.benchmark=True.
  rank_zero_warn(
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


...

Epoch 28, global step 2842: 'val/the_metric' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 29, global step 2940: 'val/the_metric' reached -4.07053 (best -4.07053), saving model to './logs/pretrained_mof_seed42_from_pmtransformer/version_7/checkpoints/epoch=29-step=2940.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

Epoch 30, global step 3038: 'val/the_metric' was not in top 1
